In [3]:
qui {

use d_multibranduncoded, clear
sort unitid year
gen name_first7 =substr(inst_name_new, 1, 7)
by unitid: gen name_first7_change=name_first7[_n+1] if name_first7!=name_first7[_n+1] & ///
systemid~=systemid[_n-1]&name_first7~=name_first7[_n+3]&name_first7[_n+3]~="" &systemid[_n-1]~=.
replace name_first7=name_first7_change if name_first7_change!=""
    
quietly for varlist type_*: recode X .=0
unab xvars:type_*
gen which_max = "" 
gen max = 0 
gen offered_types=0
    
** Calculate in what field the school offers the most degrees
quietly foreach x of local xvars { 
    replace which_max = "`x'" if `x' > max & `x'<.
    replace max = `x' if `x' > max & `x'<.
	replace offered_types=offered_types+1 if `x'>0 & `x'<.
	gen dpct`x'=`x' / total_degrees
}
replace offered_types=. if offered_types==0

quietly foreach x in type_health type_law type_culinarycosmetic ///
type_arts type_tech type_biz type_other { 
bysort systemid year: egen _brands`x'=nvals(name_first7) if which_max=="`x'"

**CODE MAXIMUM NUMBER OF BRANDS PER MODAL DEGREE FIELD FOR EACH FIRM IN A GIVEN YEAR**

bysort systemid year which_max: egen brands=nvals(name_first7)
bysort systemid year: egen maxbrands=max(brands)

**CREATE A VARIABLE FOR MULTIBRAND FIRM WITH BRAND THRESHOLD BEING 3 OR GREATER**

gen multi_brand=0
replace multi_brand=1 if maxbrands>2

**CREATE A VARIABLE FOR MULTIBRAND FIRM WITH BRAND THRESHOLD BEING 2 OR GREATER**

gen multi_brand2=0
replace multi_brand2=1 if maxbrands>1

**CREATE A VARIABLE FOR MULTIBRAND FIRM WITH BRAND THRESHOLD BEING 4 OR GREATER**

gen multi_brand4=0
replace multi_brand4=1 if maxbrands>3

bysort systemid year: egen system_under=total(all_under), missing

label var multi_brand "multi-brand"
label var multi_brand2 "2 brand threshold"
label var multi_brand4 "4 brand threshold"
label var investor "investor owned"
label var system_under "firm-level enrollment (1,000s)"
label var all_under "campus enrollment (1,000s)"
label var online "online"
label var selective "selective admissions"
label define iclevel 1 "BA offered" 2 "AA offered"
label values iclevel iclevel
label var pell_grants_per_fte_c "Pell $ per student"
label var black_share_w "% Black"
label var hisp_share_w "% Hispanic"
label var white_share_w "% White"
label var dpcttype_arts "% degr. art"
label var dpcttype_biz "% degr. biz"
label var dpcttype_culinarycosmetic "% degr. personal services"
label var dpcttype_health "% degr. health"
label var dpcttype_law "% degr. law"
label var dpcttype_tech "% degr. tech"
label var tuitionall_c_w tuition
label var loan_amount_borrower_c_w borrowing
label var ft_faculty_per_100fte_w "# faculty"
label var law_enf_frst_this_yr "law enforcement"
label var grad_rate_150_p4yr_w "BA grad rate"
label var grad_rate_150_p2yr_w "other grad rate"
label var grad_rate_150_p_w "undergrad grad rate"
quietly replace grad_rate_150_p4yr_w=grad_rate_150_p4yr_w*100 if grad_rate_150_p4yr_w!=0
quietly replace grad_rate_150_p2yr_w=grad_rate_150_p2yr_w*100 if grad_rate_150_p2yr_w!=0
quietly replace grad_rate_150_p_w=grad_rate_150_p_w*100 if grad_rate_150_p_w!=0
replace system_under =system_under/1000
replace all_under=all_under/1000

replace tuitionall_c_w = tuitionall_c_w/1000

label var tuitionall_c_w "tuition (1,000s)"

replace loan_amount_borrower_c_w = loan_amount_borrower_c_w / 1000

label var loan_amount_borrower_c_w "loans (1,000s)"

replace pell_grants_per_fte_c_w = pell_grants_per_fte_c_w/ 1000
    
label var pell_grants_per_fte_c_w "Pell $ per student (1,000s)"

keep unitid year iclevel opeid selective systemid systemnm online fte_count all_under inst_name_new ///
    loan_amount_borrower_c_w grad_rate_150_p_w grad_rate_150_p2yr_w grad_rate_150_p4yr_w ft_faculty_per_100fte_w ///
    white_share_w black_share_w hisp_share_w tuitionall_c_w pell_grants_per_fte_c_w state_n law_enf_frst_this_yr ///
    sftetotl sftesale total_degrees investor name_first7 dpcttype_health dpcttype_law ///
    dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz dpcttype_other brands ///
    multi_brand multi_brand2 multi_brand4 system_under offered_types _brands*
    
save d_unitidasymmetry, replace
    
}

In [4]:
describe


Contains data from d_unitidasymmetry.dta
  obs:        78,202                          
 vars:            40                          5 Aug 2021 16:40
 size:    28,543,730                          
--------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
--------------------------------------------------------------------------------
unitid          double  %12.0g                
year            float   %9.0g                 
iclevel         byte    %10.0g     iclevel    Highest degree offered**
opeid           str8    %9s                   Office of Postsecondary Education
                                                (OPE) ID Number
selective       float   %9.0g                 selective admissions
systemid        double  %9.0g                 
systemnm        str105  %105s                 
online          float   %9.0g                 online
fte_count     